# Flow
## 1. 모든 데이터 받아서 무조건 concat -- 1시까지
- concat 데이터셋 csv로 빼 놓기

## 2. 일단 baseline 학습

## 3. baseline 학습한 결과로 cleanlab 돌리기
- cleanlab 후에 결과 csv로 빼 놓기

## 4. 유사 문장 제거
- 유사 문장 제거 후 csv로 빼 놓기

## 5. 다시 baseline 학습

In [ ]:
pip install cleanlab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 952.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 11.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from cleanlab.filter import find_label_issues
from cleanlab.dataset import health_summary
import os
import numpy as np

# 데이터셋 불러오기

In [ ]:
path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/Data-Centric/team_data/"

In [ ]:
# 수현님 데이터셋
train_by_shyun_1 = pd.read_csv(path + "train_shyun.csv")
train_by_shyun_2 = pd.read_csv(path + "train_shyun_1.csv")
# 상의님 데이터셋
train_by_sangui_1 = pd.read_csv(path + "ascii_ratio_higher_20.csv")
train_by_sangui_2 = pd.read_csv(path + "ascii_ratio_higher_20_mlm.csv")
train_by_sangui_3 = pd.read_csv(path + "ascii_ratio_higher_20_mlm_btm.csv")
train_by_sangui_4 = pd.read_csv(path + "ascii_ratio_lower_20_rlm.csv")
train_by_sangui_5 = pd.read_csv(path + "ascii_ratio_lower_20_rlm_normal_candidates.csv")
# 정휘님 데이터셋
train_by_jeonghwi = pd.read_csv(path + "train_jeonghwi.csv")
# 민지님 데이터셋
train_by_minji = pd.read_csv(path + "train_minji.csv")

In [ ]:
train_by_sangui = pd.concat([train_by_sangui_1, train_by_sangui_2, train_by_sangui_3, train_by_sangui_4, train_by_sangui_5], axis=0, ignore_index=True)
train_by_shyun = pd.concat([train_by_shyun_1, train_by_shyun_2], axis=0, ignore_index=True)

# 잠깐 데이터 보기

In [ ]:
print(f"수현님 데이터셋 크기 : {len(train_by_shyun)}")
print(f"상의님 데이터셋 크기 : {len(train_by_sangui)}")
print(f"정휘님 데이터셋 크기 : {len(train_by_jeonghwi)}")
print(f"민지님 데이터셋 크기 : {len(train_by_minji)}")

수현님 데이터셋 크기 : 3298
상의님 데이터셋 크기 : 6319
정휘님 데이터셋 크기 : 3333
민지님 데이터셋 크기 : 2605


In [ ]:
print(f"수현님 데이터셋 컬럼 : {train_by_shyun.columns} == {len(train_by_shyun.columns)}")
print(f"상의님 데이터셋 컬럼 : {train_by_sangui.columns} == {len(train_by_sangui.columns)}")
print(f"정휘님 데이터셋 컬럼 : {train_by_jeonghwi.columns} == {len(train_by_jeonghwi.columns)}")
print(f"민지님 데이터셋 컬럼 : {train_by_minji.columns} == {len(train_by_minji.columns)}")

수현님 데이터셋 컬럼 : Index(['ID', 'text', 'target', 'polluted_lv'], dtype='object') == 4
상의님 데이터셋 컬럼 : Index(['ID', 'text', 'target', 'ascii_ratio'], dtype='object') == 4
정휘님 데이터셋 컬럼 : Index(['Unnamed: 0', 'ID', 'text', 'target'], dtype='object') == 4
민지님 데이터셋 컬럼 : Index(['ID', 'text', 'target'], dtype='object') == 3


# 데이터셋 concat

## concat 위한 데이터 프레임 컬럼 정리

In [ ]:
# 정휘님 데이터셋
train_by_jeonghwi = train_by_jeonghwi.drop(train_by_jeonghwi.columns[0], axis=1)
train_by_jeonghwi.head()

,ID,text,target
0,ynat-v1_train_01346,o국 초등학교 3부% 투Y 교육 시킨-,3
1,ynat-v1_train_00630,베스트셀러 언어의 온도 출간 7개월 만에 1위 올라,3
2,ynat-v1_train_00363,KAIST 온도따라 변하는 스마트 입자 개발,3
3,ynat-v1_train_01322,시진핑 방문에 마카오 초비상…홍콩 기자들 입경 거부당해종합,6
4,ynat-v1_train_02361,배틀그라운드 국내 서버 별도 운영…해외 이용자와 분리,4


In [ ]:
# 상의님 데이터셋
train_by_sangui = train_by_sangui.drop(columns=['ascii_ratio'])
train_by_sangui.head()

,ID,text,target
0,ynat-v1_train_00000,정i :파1 미사z KT( 이용기간 2e 단] Q분종U2보,4
1,ynat-v1_train_00001,K찰.국DLwo 로L3한N% 회장 2 T0&}송=,3
2,ynat-v1_train_00002,"m 김정) 자주통일 새,?r열1나가야1보",2
3,ynat-v1_train_00004,pI美대선I앞두고 R2fr단 발] $비해 감시 강화,6
4,ynat-v1_train_00006,프로야구~롯TKIAs광주 경기 y천취소,1


In [ ]:
# 수현님 데이터셋
train_by_shyun = train_by_shyun.drop(columns=['polluted_lv'])
train_by_shyun.head()

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,5
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,1
3,ynat-v1_train_00008,박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4


## concat

In [ ]:
train_by_team = pd.concat([train_by_shyun, train_by_sangui, train_by_jeonghwi, train_by_minji], axis=0, ignore_index=True)
train_by_team.head()

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,5
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,1
3,ynat-v1_train_00008,박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4


In [ ]:
print(f"병합 데이터셋 크기 : {len(train_by_team)}")

병합 데이터셋 크기 : 15555


In [ ]:
#train_by_team.to_csv(path + "train_team.csv")

# Cleanlab 적용 by 수현님 코드

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(DEVICE)

model_path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/Data-Centric/model"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path + "/checkpoint-682", num_labels=7).to(DEVICE)

cuda


In [ ]:
train = pd.read_csv(path + "train_team.csv")

model.eval()
preds = []
for _, data in tqdm(train.iterrows(), total=len(train)):
    tokenized = tokenizer(data['text'], padding='max_length', max_length=50, truncation=True, return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        logits = model(**tokenized).logits.cpu().numpy()
        softmax = [np.exp(x)/np.sum(np.exp(logits)) for x in logits]
        preds.append({'ID': data['ID'], 'softmax': softmax})

preds[0]

  0%|          | 0/15555 [00:00<?, ?it/s]

{'ID': 'ynat-v1_train_00003',
 'softmax': [array([0.01172413, 0.01219494, 0.00550602, 0.02566173, 0.82957345,
         0.10931693, 0.00602276], dtype=float32)]}

In [ ]:
os.environ["TOKENIZERS_PARALLELISM"] = "false"

pred_probs = [np.array(p['softmax'][0]) for p in preds]
pred_probs = np.array(pred_probs)

ordered_label_issues = find_label_issues(
    labels=train['target'],
    pred_probs=pred_probs,
    return_indices_ranked_by='self_confidence'
)

In [ ]:
print(f"total {len(ordered_label_issues)}")
for issue in ordered_label_issues:
    print('input text:', train.iloc[issue]['text'])
    print('label:', train.iloc[issue]['target'])
    pred = np.argmax(pred_probs[issue])
    print('pred:', pred)
    print('----------------------------------------')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
input text: 오S 선+R*"y안 처리O9산…경IA= 혈투 D의5P+"
label: 3
pred: 2
----------------------------------------
input text: 유영민 미래부장관 개혁 예고…새 술은 새 부대에종합
label: 3
pred: 2
----------------------------------------
input text: 김태 의원 대통령 車 에 품사 … … 출 · … … 충돌 우려 … 옥석을 가려서 못 해라
label: 5
pred: 2
----------------------------------------
input text: 정몽구 현대건설 이사직 물러나…자동차에 주력
label: 4
pred: 5
----------------------------------------
input text: 천사는병원 장료료 폐지 .. 업무에화
label: 0
pred: 3
----------------------------------------
input text: 123층 롯데월드타워 외벽 청소 개시닦을 유리창만 4만2천장
label: 4
pred: 3
----------------------------------------
input text: 아 · 티드 펜스 하우스 .. 부산 롯데에 문을 연
label: 0
pred: 1
----------------------------------------
input text: 현대증권 증권사사 분기 평균 영업익 1익 천여 억원 … 삼성 ·기로익익 증가
label: 4
pred: 5
----------------------------------------
input text: K로-구zP러리캡 위b J국E%에 T금 3/T천)00I...
label: 1
pred: 4
----------------------------------------
input text: 김해영 청년희망펀드 기부자 절반이 수탁은행 직

In [ ]:
for issue in tqdm(ordered_label_issues):
  train.loc[issue, "target"] = np.argmax(pred_probs[issue])

  0%|          | 0/1534 [00:00<?, ?it/s]

In [ ]:
train[train["text"].str.contains("부산국제영화제 23회 개막")] # pred -> 0

,Unnamed: 0,ID,text,target
15273,15273,synthetic_llama_30090,부산국제영화제 23회 개막,3


In [ ]:
train[train["text"].str.contains("부산국제영화제 23회 개막")] # pred -> 0

,Unnamed: 0,ID,text,target
15273,15273,synthetic_llama_30090,부산국제영화제 23회 개막,0


In [ ]:
#train.to_csv(path + "train_team_cleanlab.csv")

In [ ]:
class_names = range(7)
health_summary(train['target'], pred_probs, class_names=class_names)

-----------------------------------------------------------
|  Generating a Cleanlab Dataset Health Summary           |
|   for your dataset with 15,555 examples and 7 classes.  |
|  Note, Cleanlab is not a medical doctor... yet.         |
-----------------------------------------------------------

Overall Class Quality and Noise across your dataset (below)
------------------------------------------------------------ 



,Class Name,Class Index,Label Issues,Inverse Label Issues,Label Noise,Inverse Label Noise,Label Quality Score
0,3,3,523,312,0.222269,0.145658,0.777731
1,0,0,230,353,0.112360,0.162673,0.887640
2,5,5,203,208,0.094727,0.096834,0.905273
3,2,2,141,133,0.069733,0.066038,0.930267
4,4,4,165,269,0.068126,0.106492,0.931874
5,6,6,150,126,0.066964,0.056859,0.933036
6,1,1,122,133,0.052405,0.056862,0.947595



Class Overlap. In some cases, you may want to merge classes in the top rows (below)
-----------------------------------------------------------------------------------



,Class Name A,Class Name B,Class Index A,Class Index B,Num Overlapping Examples,Joint Probability
0,0,3,0,3,390,0.025072
1,4,5,4,5,164,0.010543
2,3,4,3,4,137,0.008807
3,3,5,3,5,112,0.007200
4,2,6,2,6,98,0.006300
5,2,3,2,3,91,0.005850
6,0,1,0,1,65,0.004179
7,1,3,1,3,58,0.003729
8,3,6,3,6,47,0.003022
9,0,5,0,5,42,0.002700



 * Overall, about 8% (1,187 of the 15,555) labels in your dataset have potential issues.
 ** The overall label health score for this dataset is: 0.92.

Generated with <3 from Cleanlab.



{'overall_label_health_score': 0.9236901317904211,
 'joint': array([[1.16811315e-01, 2.76438444e-03, 6.42880103e-04, 7.07168113e-03,
         1.47862424e-03, 1.99292832e-03, 8.35744134e-04],
        [1.41433623e-03, 1.41819351e-01, 8.35744134e-04, 2.63580842e-03,
         1.02860816e-03, 1.54291225e-03, 3.85728062e-04],
        [5.78592093e-04, 8.35744134e-04, 1.20925747e-01, 2.25008036e-03,
         1.02860816e-03, 1.09289617e-03, 3.27868852e-03],
        [1.80006429e-02, 1.09289617e-03, 3.60012858e-03, 1.17647059e-01,
         5.33590485e-03, 3.79299261e-03, 1.80006429e-03],
        [1.02860816e-03, 1.54291225e-03, 6.42880103e-05, 3.47155256e-03,
         1.45098039e-01, 3.27868852e-03, 1.22147220e-03],
        [7.07168113e-04, 7.07168113e-04, 3.85728062e-04, 3.40726455e-03,
         7.26454516e-03, 1.24718740e-01, 5.78592093e-04],
        [9.64320154e-04, 1.60720026e-03, 3.02153648e-03, 1.22147220e-03,
         1.15718419e-03, 1.67148827e-03, 1.34361941e-01]]),
 'classes_by_label_qu

In [ ]:
train = train.drop(train.columns[0], axis=1)
train.head()

,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,5
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,1
3,ynat-v1_train_00008,박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4


# 유사 문장 확인

In [ ]:
duplicates = train[train['text'].duplicated(keep=False)]
print(len(duplicates))
duplicates

5767


,ID,text,target
5,ynat-v1_train_00011,NH투자 1월 옵션 만기일 매도 우세,5
6,ynat-v1_train_00012,황총리 각 부처 비상대비태세 철저히 강구해야,2
7,ynat-v1_train_00016,공사업체 협박에 분쟁해결 명목 돈 받은 언론인 집행유예,3
9,ynat-v1_train_00019,페이스북 인터넷 드론 아퀼라 실물 첫 시험비행 성공,4
13,ynat-v1_train_00034,안보리 대북결의안 2270호 이행보고서 제출한 나라 70개 육박,6
...,...,...,...
14937,ynat-v1_train_00326,지구 임계온도 1.5도 목표 촉구 1인 시위,3
14938,ynat-v1_train_00951,美매체 여성들이 김정은 싫어하는 이유 분석,6
14939,ynat-v1_train_01754,美 미사일방어시스템 한국에 최대한 조기구축 희망1보,6
14967,synthetic_llama_00035,기다렸다 눈과 추위야…강원 스키장 개장 준비 분주,0


In [ ]:
from itertools import islice

duplicates_grouped = train[train['text'].duplicated(keep=False)].groupby('text')

print(f"겹치는 데이터 총 개수 : {len(duplicates_grouped)}")

# 그룹별로 출력
for sentence, group in islice(duplicates_grouped, 50):
    print(f"문장: {sentence}")
    print(group)
    print("\n" + "-"*50 + "\n")

겹치는 데이터 총 개수 : 2038
문장:   
                       ID text  target
7739  ynat-v1_train_01959            1
7885  ynat-v1_train_02222            1

--------------------------------------------------

문장:  G 김현수 두산=y 9k @i인 투2포
                        ID                    text  target
2981   ynat-v1_train_01926   G 김현수 두산=y 9k @i인 투2포       1
4421   ynat-v1_train_01926   G 김현수 두산=y 9k @i인 투2포       1
12754  ynat-v1_train_01926   G 김현수 두산=y 9k @i인 투2포       1

--------------------------------------------------

문장:  Q산U 림자금\ I,SC다…Y감원장 b합YM시스템_구6
                        ID                             text  target
4926   ynat-v1_train_02766   Q산U 림자금\ I,SC다…Y감원장 b합YM시스템_구6       5
12942  ynat-v1_train_02766   Q산U 림자금\ I,SC다…Y감원장 b합YM시스템_구6       5

--------------------------------------------------

문장:  당대5 김정은"#Q 결산보고 'h'내O
                        ID                    text  target
4510   ynat-v1_train_02057   당대5 김정은"#Q 결산보고 'h'내O       2
12797  ynat-v1_train_02057   당대5 김정은"#Q 결산보고 'h'내

### ID, text, target이 모두 같은 데이터셋은 합치기

In [ ]:
train = train.drop_duplicates(subset=['ID', 'text', 'target'], keep='first').reset_index(drop=True)

In [ ]:
duplicates = train[train['text'].duplicated(keep=False)]
print(len(duplicates))

422


In [ ]:
from itertools import islice

duplicates_grouped = train[train['text'].duplicated(keep=False)].groupby('text')

print(f"겹치는 데이터 총 개수 : {len(duplicates_grouped)}")

# 그룹별로 출력
for sentence, group in islice(duplicates_grouped, 50):
    print(f"문장: {sentence}")
    print(group)
    print("\n" + "-"*50 + "\n")

겹치는 데이터 총 개수 : 201
문장:   
                       ID text  target
7646  ynat-v1_train_01959            1
7790  ynat-v1_train_02222            1

--------------------------------------------------

문장:  시판^주금공 한국의 주e-융 70'TBa
                        ID                     text  target
2491   ynat-v1_train_00489   시판^주금공 한국의 주e-융 70'TBa       5
10306  ynat-v1_train_00489   시판^주금공 한국의 주e-융 70'TBa       3

--------------------------------------------------

문장: "타&후의.5J^9지c 이{라엘 야권연대9성사SD
                        ID                         text  target
3865   ynat-v1_train_00998  "타&후의.5J^9지c 이{라엘 야권연대9성사SD       6
10311  ynat-v1_train_00998  "타&후의.5J^9지c 이{라엘 야권연대9성사SD       2

--------------------------------------------------

문장: ##용용
                       ID  text  target
7940  ynat-v1_train_02469  ##용용       0
8017  ynat-v1_train_02611  ##용용       6

--------------------------------------------------

문장: &0L0년이면x-공지능L_건강·P률 상담
                        ID                    text  targe

### ID만 다른 경우 합치기


In [ ]:
train = train.drop_duplicates(subset=['text', 'target'], keep='first').reset_index(drop=True)

duplicates = train[train['text'].duplicated(keep=False)]

print(len(duplicates))

399


In [ ]:
duplicates.head()

,ID,text,target
15,ynat-v1_train_00036,크루즈 관광객용 반나절 부산 해안 트레킹 상품 개발,0
36,ynat-v1_train_00093,문대통령 호주 총리와 축구 평가전 1대1 무승부 축하,1
44,ynat-v1_train_00111,세아트 드라이빙 시뮬레이터,4
61,ynat-v1_train_00151,한국형 발사체 75t 엔진 첫 연소시험 성공,4
81,ynat-v1_train_00189,400년 전 일본 건너간 조선 첫 여성도공 백파선,0


In [ ]:
from itertools import islice

duplicates_grouped = train[train['text'].duplicated(keep=False)].groupby('text')

print(f"겹치는 데이터 총 개수 : {len(duplicates_grouped)}")

# 그룹별로 출력
for sentence, group in islice(duplicates_grouped, 50):
    print(f"문장: {sentence}")
    print(group)
    print("\n" + "-"*50 + "\n")

겹치는 데이터 총 개수 : 190
문장:  시판^주금공 한국의 주e-융 70'TBa
                        ID                     text  target
2487   ynat-v1_train_00489   시판^주금공 한국의 주e-융 70'TBa       5
10296  ynat-v1_train_00489   시판^주금공 한국의 주e-융 70'TBa       3

--------------------------------------------------

문장: "타&후의.5J^9지c 이{라엘 야권연대9성사SD
                        ID                         text  target
3861   ynat-v1_train_00998  "타&후의.5J^9지c 이{라엘 야권연대9성사SD       6
10301  ynat-v1_train_00998  "타&후의.5J^9지c 이{라엘 야권연대9성사SD       2

--------------------------------------------------

문장: ##용용
                       ID  text  target
7933  ynat-v1_train_02469  ##용용       0
8010  ynat-v1_train_02611  ##용용       6

--------------------------------------------------

문장: &0L0년이면x-공지능L_건강·P률 상담
                        ID                    text  target
4375   ynat-v1_train_01939  &0L0년이면x-공지능L_건강·P률 상담       4
10308  ynat-v1_train_01939  &0L0년이면x-공지능L_건강·P률 상담       3

--------------------------------------------------

문장: 

### target 값이 다른 데이터셋 삭제

In [ ]:
test_train = train.copy()
print(len(test_train))
test_train.head()

12035


,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,5
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,1
3,ynat-v1_train_00008,박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4


In [ ]:
# 'id'와 'text'가 같고 'target'이 다른 경우, 하나의 행만 남기기
test_train = test_train.drop_duplicates(subset=['ID', 'text'], keep='first').reset_index(drop=True)
print(len(test_train))
test_train.head()

11829


,ID,text,target
0,ynat-v1_train_00003,갤노트8 주말 27만대 개통시장은 불법 보조금 얼룩,4
1,ynat-v1_train_00005,미성인 6명 중 1명꼴 배우자연인 빚 떠안은 적 있다,5
2,ynat-v1_train_00007,아가메즈 33득점 우리카드 KB손해보험 완파3위 굳,1
3,ynat-v1_train_00008,박대통령 얼마나 많이 놀라셨어요경주 지진현장 방문종합,2
4,ynat-v1_train_00009,듀얼심 아이폰 하반기 출시설 솔솔알뜰폰 기대감,4


In [ ]:
test_train.to_csv(path + "train_team_cleanlab_cut_similarity.csv")

In [ ]:
from itertools import islice

duplicates_grouped = test_train[test_train['text'].duplicated(keep=False)].groupby('text')

print(f"겹치는 데이터 총 개수 : {len(duplicates_grouped)}")

# 그룹별로 출력
for sentence, group in islice(duplicates_grouped, 50):
    print(f"문장: {sentence}")
    print(group)
    print("\n" + "-"*50 + "\n")

겹치는 데이터 총 개수 : 3
문장: ##용용
                       ID  text  target
7927  ynat-v1_train_02469  ##용용       0
8004  ynat-v1_train_02611  ##용용       6

--------------------------------------------------

문장: ··
                       ID text  target
7382  ynat-v1_train_01476   ··       1
7938  ynat-v1_train_02484   ··       4

--------------------------------------------------

문장: 쿠쿠홈시스 최대주주 쿠쿠홀딩스로 변경
                            ID                  text  target
902        ynat-v1_train_01861  쿠쿠홈시스 최대주주 쿠쿠홀딩스로 변경       5
9028  augmented-v1_train_01861  쿠쿠홈시스 최대주주 쿠쿠홀딩스로 변경       4

--------------------------------------------------



# target 같은 값 다시 relabeling by model_cleanlab

In [ ]:
duplicates.head()

,ID,text,target
15,ynat-v1_train_00036,크루즈 관광객용 반나절 부산 해안 트레킹 상품 개발,0
36,ynat-v1_train_00093,문대통령 호주 총리와 축구 평가전 1대1 무승부 축하,1
44,ynat-v1_train_00111,세아트 드라이빙 시뮬레이터,4
61,ynat-v1_train_00151,한국형 발사체 75t 엔진 첫 연소시험 성공,4
81,ynat-v1_train_00189,400년 전 일본 건너간 조선 첫 여성도공 백파선,0


In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(DEVICE)

model_path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/Data-Centric/model_1"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path + "/checkpoint-682", num_labels=7).to(DEVICE)

new_train = duplicates.copy()

model.eval()

preds = []

for _, data in tqdm(new_train.iterrows(), total=len(new_train)):
    tokenized = tokenizer(data['text'], padding='max_length', max_length=50, truncation=True, return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        logits = model(**tokenized).logits.cpu().numpy()
        softmax = [np.exp(x)/np.sum(np.exp(logits)) for x in logits]
        preds.append({'ID': data['ID'], 'softmax': softmax})

os.environ["TOKENIZERS_PARALLELISM"] = "false"

pred_probs = [np.array(p['softmax'][0]) for p in preds]
pred_probs = np.array(pred_probs)

ordered_label_issues = find_label_issues(
    labels=new_train['target'],
    pred_probs=pred_probs,
    return_indices_ranked_by='self_confidence'
)

print(f"total {len(ordered_label_issues)}")
for issue in ordered_label_issues:
    print('input text:', new_train.iloc[issue]['text'])
    print('label:', new_train.iloc[issue]['target'])
    pred = np.argmax(pred_probs[issue])
    print('pred:', pred)
    print('----------------------------------------')


cuda


  0%|          | 0/399 [00:00<?, ?it/s]

total 196
input text: 중러 원자로 건설 2조원 계약 맺는다
label: 1
pred: 6
----------------------------------------
input text: 네이Du Ln데이E센터 9q?]1 본선 진출 L0개s 선정
label: 1
pred: 4
----------------------------------------
input text: 직통령 펭수 UAR로 만난다…3D 영상·입체 스티커 나와
label: 1
pred: 3
----------------------------------------
input text: 카드 IC단말기 전환율 98.5%…미전환 단말기 보안 조치
label: 1
pred: 4
----------------------------------------
input text: 롯데그룹 글로벌 진출 지도 바뀐다
label: 1
pred: 5
----------------------------------------
input text: 정부 거점항만으로 한진해운 선박 유도해 화물 하역
label: 1
pred: 5
----------------------------------------
input text: 힐러리 사우스캐롤라이나 경선서 샌더스 꺾고 승리2보
label: 1
pred: 2
----------------------------------------
input text: 기업은행 IBK희망디자X 대구t대신동 *말골목 지원
label: 1
pred: 3
----------------------------------------
input text: 대한항공 인천∼바르셀로나 노선 첫 취항
label: 4
pred: 3
----------------------------------------
input text: 롯데호텔 대표이사 내정자 김현식 전무
label: 1
pred: 5
----------------------------------------
input text: ##용용
label:

In [ ]:
for issue in tqdm(ordered_label_issues):
  new_train.loc[issue, "target"] = np.argmax(pred_probs[issue])

  0%|          | 0/196 [00:00<?, ?it/s]

In [ ]:
len(duplicates)

399

In [ ]:
duplicates = duplicates.drop_duplicates(subset=['text'], keep='first').reset_index(drop=True)
len(duplicates)

190

In [ ]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

print(DEVICE)

model_path = "/content/drive/MyDrive/네이버 부스트 캠프/프로젝트/Level2_프로젝트/Data-Centric/model_1"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path + "/checkpoint-682", num_labels=7).to(DEVICE)

new_train = duplicates.copy()

model.eval()

preds = []

for _, data in tqdm(new_train.iterrows(), total=len(new_train)):
    tokenized = tokenizer(data['text'], padding='max_length', max_length=50, truncation=True, return_tensors='pt').to(DEVICE)
    with torch.no_grad():
        logits = model(**tokenized).logits.cpu().numpy()
        softmax = [np.exp(x)/np.sum(np.exp(logits)) for x in logits]
        preds.append({'ID': data['ID'], 'softmax': softmax})

os.environ["TOKENIZERS_PARALLELISM"] = "false"

pred_probs = [np.array(p['softmax'][0]) for p in preds]
pred_probs = np.array(pred_probs)

ordered_label_issues = find_label_issues(
    labels=new_train['target'],
    pred_probs=pred_probs,
    return_indices_ranked_by='self_confidence'
)

print(f"total {len(ordered_label_issues)}")
for issue in ordered_label_issues:
    print('input text:', new_train.iloc[issue]['text'])
    print('label:', new_train.iloc[issue]['target'])
    pred = np.argmax(pred_probs[issue])
    print('pred:', pred)
    print('----------------------------------------')


cuda


  0%|          | 0/190 [00:00<?, ?it/s]

total 80
input text: 카드 IC단말기 전환율 98.5%…미전환 단말기 보안 조치
label: 1
pred: 4
----------------------------------------
input text: 힐러리 사우스캐롤라이나 경선서 샌더스 꺾고 승리2보
label: 1
pred: 2
----------------------------------------
input text: 대한항공 인천∼바르셀로나 노선 첫 취항
label: 4
pred: 3
----------------------------------------
input text: 롯데호텔 대표이사 내정자 김현식 전무
label: 1
pred: 5
----------------------------------------
input text: 중국인 래퍼 인터넷 생방송 중 손가락 자해 행동 논란
label: 1
pred: 3
----------------------------------------
input text: 조u사 없는 에어On# a^트리아/Ib 시범f행
label: 6
pred: 4
----------------------------------------
input text: 올림픽 펜스와 아베 뒤로 김영남과 김여정
label: 1
pred: 6
----------------------------------------
input text: 비닐봉지 대신 바나나잎 포장…베트남 마트의 친환경 실험
label: 4
pred: 3
----------------------------------------
input text: 정부 거점항만으로 한진해운 선박 유도해 화물 하역
label: 6
pred: 5
----------------------------------------
input text: 카Aj서$#로야S d보 본다
label: 4
pred: 1
----------------------------------------
input text: 만트럭버스코리아 페어 2018 기자

In [ ]:
for issue in tqdm(ordered_label_issues):
  new_train.loc[issue, "target"] = np.argmax(pred_probs[issue])

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
from itertools import islice

duplicates_grouped = new_train[new_train['text'].duplicated(keep=False)].groupby('text')

print(f"겹치는 데이터 총 개수 : {len(duplicates_grouped)}")

# 그룹별로 출력
for sentence, group in islice(duplicates_grouped, 50):
    print(f"문장: {sentence}")
    print(group)
    print("\n" + "-"*50 + "\n")

겹치는 데이터 총 개수 : 0


In [ ]:
len(new_train)

190

In [ ]:
test_train = pd.concat([test_train, new_train], axis=0, ignore_index=True)
test_train.to_csv(path + "train_cleanlab_2.csv")

In [ ]:
len(test_train)

12019